# POS_Tagging, Viterbi, Forward,Backward Procedure

In [1]:
import os
import sys
import nltk
import math
import time
import re
import numpy as np
from operator import itemgetter

## Reading Training File

In [2]:
cwd =  os.getcwd()
inputfile = open(cwd+'/Brown_tagged_train.txt','r')
brown = inputfile.readlines()

## Dictionary containing words and respective tags

In [3]:
brown_tags = []
brown_words = []
for sentence in brown:
    words = sentence.strip().split()
    list_tag = []
    for wordtag in words:
        tag = wordtag.rsplit('/',1)[-1]
        list_tag.append(tag)
    brown_tags.append(list_tag)

for sent in brown_tags:
    sent.insert(0,'*')
    sent.insert(0,'*')
    sent.append('STOP')

for sentence in brown:
    words = sentence.strip().split()
    list_word = []
    for wordtag in words:
        word = wordtag.rsplit('/',1)[0]
        word = word.lower()
        list_word.append(word)
    brown_words.append(list_word)

for sent in brown_words:
    sent.insert(0,'*')
    sent.insert(0,'*')
    sent.append('STOP')

brown_words


[['*',
  '*',
  'at',
  'that',
  'time',
  'highway',
  'engineers',
  'traveled',
  'rough',
  'and',
  'dirty',
  'roads',
  'to',
  'accomplish',
  'their',
  'duties',
  '.',
  'STOP'],
 ['*',
  '*',
  'using',
  'privately-owned',
  'vehicles',
  'was',
  'a',
  'personal',
  'hardship',
  'for',
  'such',
  'employees',
  ',',
  'and',
  'the',
  'matter',
  'of',
  'providing',
  'state',
  'transportation',
  'was',
  'felt',
  'perfectly',
  'justifiable',
  '.',
  'STOP'],
 ['*',
  '*',
  'once',
  'the',
  'principle',
  'was',
  'established',
  ',',
  'the',
  'increase',
  'in',
  'state-owned',
  'vehicles',
  'came',
  'rapidly',
  '.',
  'STOP'],
 ['*',
  '*',
  'and',
  'reasons',
  'other',
  'than',
  'employee',
  'need',
  'contributed',
  'to',
  'the',
  'growth',
  '.',
  'STOP'],
 ['*',
  '*',
  'table',
  '1',
  'immediately',
  'below',
  'shows',
  'the',
  'rate',
  'of',
  'growth',
  'of',
  'vehicles',
  'and',
  'employees',
  '.',
  'STOP'],
 ['*',
 

## unigram count of tags

In [4]:
unigram_count = dict()
unigram_tags = []
for sent_tags in brown_tags:
    sent_tags = sent_tags
    list_unigram = []
    for i in range(len(sent_tags)):
        unigram = tuple(sent_tags[i:i+1])
        list_unigram.append(unigram)
    unigram_tags.append(list_unigram)
unigram_tags

[[('*',),
  ('*',),
  ('ADP',),
  ('DET',),
  ('NOUN',),
  ('NOUN',),
  ('NOUN',),
  ('VERB',),
  ('ADJ',),
  ('CONJ',),
  ('ADJ',),
  ('NOUN',),
  ('PRT',),
  ('VERB',),
  ('DET',),
  ('NOUN',),
  ('.',),
  ('STOP',)],
 [('*',),
  ('*',),
  ('VERB',),
  ('ADJ',),
  ('NOUN',),
  ('VERB',),
  ('DET',),
  ('ADJ',),
  ('NOUN',),
  ('ADP',),
  ('ADJ',),
  ('NOUN',),
  ('.',),
  ('CONJ',),
  ('DET',),
  ('NOUN',),
  ('ADP',),
  ('VERB',),
  ('NOUN',),
  ('NOUN',),
  ('VERB',),
  ('VERB',),
  ('ADV',),
  ('ADJ',),
  ('.',),
  ('STOP',)],
 [('*',),
  ('*',),
  ('ADP',),
  ('DET',),
  ('NOUN',),
  ('VERB',),
  ('VERB',),
  ('.',),
  ('DET',),
  ('NOUN',),
  ('ADP',),
  ('ADJ',),
  ('NOUN',),
  ('VERB',),
  ('ADV',),
  ('.',),
  ('STOP',)],
 [('*',),
  ('*',),
  ('CONJ',),
  ('NOUN',),
  ('ADJ',),
  ('ADP',),
  ('NOUN',),
  ('NOUN',),
  ('VERB',),
  ('ADP',),
  ('DET',),
  ('NOUN',),
  ('.',),
  ('STOP',)],
 [('*',),
  ('*',),
  ('NOUN',),
  ('NUM',),
  ('ADV',),
  ('ADV',),
  ('VERB',),
  ('DE

In [5]:
for unigrams in unigram_tags:
    for unigram in unigrams:
        if unigram not in unigram_count.keys():
            unigram_count[unigram] = 0
        unigram_count[unigram] += 1
unigram_count

{('*',): 54982,
 ('.',): 71555,
 ('ADJ',): 36466,
 ('ADP',): 65861,
 ('ADV',): 27683,
 ('CONJ',): 17779,
 ('DET',): 63164,
 ('NOUN',): 120770,
 ('NUM',): 6664,
 ('PRON',): 27229,
 ('PRT',): 15622,
 ('STOP',): 27491,
 ('VERB',): 89766,
 ('X',): 590}

## bigram count of tags

In [6]:
bigram_count = dict()
bigram_tags = []
for sent_tags in brown_tags:
    sent_tags = sent_tags
    list_bigram = []
    for i in range(len(sent_tags)-1):
        bigram = tuple(sent_tags[i:i+2])
        list_bigram.append(bigram)
    bigram_tags.append(list_bigram)
bigram_tags

[[('*', '*'),
  ('*', 'ADP'),
  ('ADP', 'DET'),
  ('DET', 'NOUN'),
  ('NOUN', 'NOUN'),
  ('NOUN', 'NOUN'),
  ('NOUN', 'VERB'),
  ('VERB', 'ADJ'),
  ('ADJ', 'CONJ'),
  ('CONJ', 'ADJ'),
  ('ADJ', 'NOUN'),
  ('NOUN', 'PRT'),
  ('PRT', 'VERB'),
  ('VERB', 'DET'),
  ('DET', 'NOUN'),
  ('NOUN', '.'),
  ('.', 'STOP')],
 [('*', '*'),
  ('*', 'VERB'),
  ('VERB', 'ADJ'),
  ('ADJ', 'NOUN'),
  ('NOUN', 'VERB'),
  ('VERB', 'DET'),
  ('DET', 'ADJ'),
  ('ADJ', 'NOUN'),
  ('NOUN', 'ADP'),
  ('ADP', 'ADJ'),
  ('ADJ', 'NOUN'),
  ('NOUN', '.'),
  ('.', 'CONJ'),
  ('CONJ', 'DET'),
  ('DET', 'NOUN'),
  ('NOUN', 'ADP'),
  ('ADP', 'VERB'),
  ('VERB', 'NOUN'),
  ('NOUN', 'NOUN'),
  ('NOUN', 'VERB'),
  ('VERB', 'VERB'),
  ('VERB', 'ADV'),
  ('ADV', 'ADJ'),
  ('ADJ', '.'),
  ('.', 'STOP')],
 [('*', '*'),
  ('*', 'ADP'),
  ('ADP', 'DET'),
  ('DET', 'NOUN'),
  ('NOUN', 'VERB'),
  ('VERB', 'VERB'),
  ('VERB', '.'),
  ('.', 'DET'),
  ('DET', 'NOUN'),
  ('NOUN', 'ADP'),
  ('ADP', 'ADJ'),
  ('ADJ', 'NOUN'),
  ('NOUN'

In [7]:
for bigrams in bigram_tags:
    for bigram in bigrams:
        if bigram not in bigram_count.keys():
            bigram_count[bigram] = 0
        bigram_count[bigram] += 1
bigram_count

{('*', '*'): 27491,
 ('*', '.'): 3513,
 ('*', 'ADJ'): 745,
 ('*', 'ADP'): 2931,
 ('*', 'ADV'): 2529,
 ('*', 'CONJ'): 1009,
 ('*', 'DET'): 5315,
 ('*', 'NOUN'): 3472,
 ('*', 'NUM'): 426,
 ('*', 'PRON'): 5336,
 ('*', 'PRT'): 1165,
 ('*', 'VERB'): 1035,
 ('*', 'X'): 15,
 ('.', '.'): 7024,
 ('.', 'ADJ'): 1837,
 ('.', 'ADP'): 4375,
 ('.', 'ADV'): 3392,
 ('.', 'CONJ'): 5203,
 ('.', 'DET'): 4762,
 ('.', 'NOUN'): 5356,
 ('.', 'NUM'): 910,
 ('.', 'PRON'): 3918,
 ('.', 'PRT'): 1763,
 ('.', 'STOP'): 27490,
 ('.', 'VERB'): 5443,
 ('.', 'X'): 82,
 ('ADJ', '.'): 4085,
 ('ADJ', 'ADJ'): 2075,
 ('ADJ', 'ADP'): 3348,
 ('ADJ', 'ADV'): 436,
 ('ADJ', 'CONJ'): 1410,
 ('ADJ', 'DET'): 227,
 ('ADJ', 'NOUN'): 23110,
 ('ADJ', 'NUM'): 198,
 ('ADJ', 'PRON'): 180,
 ('ADJ', 'PRT'): 724,
 ('ADJ', 'VERB'): 656,
 ('ADJ', 'X'): 17,
 ('ADP', '.'): 533,
 ('ADP', 'ADJ'): 5087,
 ('ADP', 'ADP'): 1375,
 ('ADP', 'ADV'): 1079,
 ('ADP', 'CONJ'): 141,
 ('ADP', 'DET'): 30604,
 ('ADP', 'NOUN'): 15902,
 ('ADP', 'NUM'): 1863,
 ('ADP'

## Trigram count of tags

In [8]:
trigram_count = dict()
trigram_tags = []
for sent_tags in brown_tags:
    list_trigram = []
    for i in range(len(sent_tags)-2):
        trigram = tuple(sent_tags[i:i+3])
        list_trigram.append(trigram)
    trigram_tags.append(list_trigram)

In [9]:
for trigrams in trigram_tags:
    for trigram in trigrams:
        if trigram not in trigram_count.keys():
            trigram_count[trigram] = 0
        trigram_count[trigram] += 1
trigram_count

{('DET', 'ADP', 'PRON'): 84,
 ('DET', 'NUM', 'VERB'): 45,
 ('CONJ', 'NUM', 'ADV'): 6,
 ('DET', 'ADJ', 'VERB'): 291,
 ('X', 'PRT', '.'): 1,
 ('ADP', 'PRON', 'CONJ'): 149,
 ('NOUN', 'NOUN', 'X'): 2,
 ('PRT', 'NUM', 'NOUN'): 37,
 ('DET', 'CONJ', 'PRT'): 6,
 ('PRON', 'PRT', 'NOUN'): 8,
 ('X', 'NOUN', 'NOUN'): 2,
 ('NUM', 'ADJ', 'PRON'): 1,
 ('VERB', 'VERB', 'VERB'): 2242,
 ('PRON', 'ADP', 'NUM'): 29,
 ('NOUN', 'ADP', 'CONJ'): 41,
 ('VERB', 'DET', 'CONJ'): 9,
 ('ADV', '.', 'VERB'): 474,
 ('CONJ', 'NUM', 'PRON'): 4,
 ('CONJ', 'CONJ', 'VERB'): 3,
 ('PRON', 'ADJ', 'ADJ'): 4,
 ('ADV', 'VERB', 'DET'): 1063,
 ('CONJ', 'ADJ', 'ADJ'): 115,
 ('ADP', 'VERB', 'X'): 1,
 ('DET', '.', 'ADJ'): 68,
 ('.', 'ADV', 'CONJ'): 20,
 ('PRT', 'DET', 'X'): 1,
 ('*', 'NUM', 'ADJ'): 33,
 ('ADJ', 'PRON', 'DET'): 1,
 ('CONJ', 'DET', 'VERB'): 218,
 ('CONJ', 'NOUN', '.'): 1102,
 ('NOUN', 'CONJ', 'PRT'): 116,
 ('*', 'ADP', 'NOUN'): 575,
 ('DET', 'VERB', 'PRT'): 98,
 ('VERB', 'CONJ', 'VERB'): 802,
 ('ADV', 'ADP', 'VERB'): 1

## Probability of unigrams,bigrams,trigrams tags

In [10]:
unigram_total = sum(unigram_count.values())
unigram_prob = dict()
for i in unigram_count:
    temp = unigram_count[i]
    temp_log = math.log(temp,2)
    temp_total = math.log(unigram_total,2)
    if i not in unigram_prob.keys():
            unigram_prob[i] = 0
    unigram_prob[i] = temp_log-temp_total
unigram_prob

{('*',): -3.5082599541831296,
 ('.',): -3.128166760108222,
 ('ADJ',): -4.100667383426458,
 ('ADP',): -3.2477949235523234,
 ('ADV',): -4.498219044842578,
 ('CONJ',): -5.137045160368816,
 ('DET',): -3.30811680485583,
 ('NOUN',): -2.37302912093957,
 ('NUM',): -6.552759034985128,
 ('PRON',): -4.522075341215421,
 ('PRT',): -5.32364017506951,
 ('STOP',): -4.50825995418313,
 ('VERB',): -2.8010502312432486,
 ('X',): -10.050360576101472}

In [15]:
bigram_total = sum(bigram_count.values())
bigram_prob = dict()
for i in bigram_count:
    temp = bigram_count[i]
    print(i)
    temp_log = math.log(temp,2)
    temp_total = math.log(bigram_total,2)
    if i not in bigram_prob.keys():
            bigram_prob[i] = 0
    bigram_prob[i] = temp_log-temp_total


('ADP', 'CONJ')
('PRON', 'ADJ')
('*', '.')
('NUM', 'NUM')
('NUM', 'PRT')
('ADP', 'NOUN')
('NUM', 'X')
('DET', 'ADP')
('ADJ', 'ADV')
('CONJ', 'DET')
('.', 'PRON')
('ADV', 'NOUN')
('ADP', 'ADP')
('*', '*')
('PRT', 'ADJ')
('.', 'ADP')
('CONJ', 'ADJ')
('ADJ', 'PRT')
('CONJ', '.')
('DET', 'PRON')
('X', 'PRON')
('X', 'DET')
('ADV', 'ADJ')
('NOUN', 'PRON')
('NUM', 'NOUN')
('.', 'STOP')
('X', 'ADP')
('ADJ', '.')
('NOUN', 'ADP')
('VERB', '.')
('.', 'ADV')
('ADJ', 'DET')
('PRON', 'DET')
('CONJ', 'CONJ')
('VERB', 'DET')
('PRON', 'NOUN')
('NOUN', 'VERB')
('ADP', 'ADJ')
('NUM', 'VERB')
('.', 'PRT')
('VERB', 'X')
('NOUN', 'NUM')
('NOUN', 'X')
('PRON', '.')
('ADV', 'ADV')
('*', 'DET')
('VERB', 'VERB')
('VERB', 'ADV')
('CONJ', 'NOUN')
('DET', 'CONJ')
('ADJ', 'X')
('NOUN', 'CONJ')
('.', 'NUM')
('PRT', '.')
('.', 'VERB')
('PRT', 'NOUN')
('ADV', 'VERB')
('NUM', 'PRON')
('*', 'CONJ')
('DET', 'PRT')
('VERB', 'NOUN')
('DET', 'ADV')
('NUM', 'ADP')
('DET', 'VERB')
('ADJ', 'CONJ')
('ADP', 'X')
('PRON', 'NUM')


In [12]:
trigram_total = sum(trigram_count.values())
trigram_prob = dict()
for i in trigram_count:
    temp = trigram_count[i]
    temp_log = math.log(temp,2)
    temp_total = math.log(trigram_total,2)
    if i not in trigram_prob.keys():
            trigram_prob[i] = 0
    trigram_prob[i] = temp_log-temp_total
trigram_prob

{('DET', 'ADP', 'PRON'): -12.729903930332021,
 ('DET', 'NUM', 'VERB'): -13.630368256781107,
 ('CONJ', 'NUM', 'ADV'): -16.537258852389627,
 ('DET', 'ADJ', 'VERB'): -10.937346010202496,
 ('X', 'PRT', '.'): -19.12222135311078,
 ('ADP', 'PRON', 'CONJ'): -11.90305283264862,
 ('NOUN', 'NOUN', 'X'): -18.12222135311078,
 ('PRT', 'NUM', 'NOUN'): -13.912767987481832,
 ('DET', 'CONJ', 'PRT'): -16.537258852389627,
 ('PRON', 'PRT', 'NOUN'): -16.12222135311078,
 ('X', 'NOUN', 'NOUN'): -18.12222135311078,
 ('NUM', 'ADJ', 'PRON'): -19.12222135311078,
 ('VERB', 'VERB', 'VERB'): -7.9916507903053535,
 ('PRON', 'ADP', 'NUM'): -14.264240357983208,
 ('NOUN', 'ADP', 'CONJ'): -13.764669348492697,
 ('ADV', 'PRON', 'ADP'): -19.12222135311078,
 ('X', '.', 'ADP'): -15.952296351668469,
 ('CONJ', 'NUM', 'PRON'): -17.12222135311078,
 ('CONJ', 'CONJ', 'VERB'): -17.537258852389623,
 ('PRON', 'ADJ', 'ADJ'): -17.12222135311078,
 ('NUM', 'ADJ', 'CONJ'): -15.537258852389625,
 ('ADV', 'VERB', 'DET'): -9.068295471579676,
 (

In [13]:
len(trigram_prob)

1560

## Implementation of Deleted Interpolation

In [14]:
def deleted_interpolation():
    lambda1 = 0
    lambda2 = 0
    lambda3 = 0
    total_sum = sum(unigram_count.values())-1
    for tag1,tag2,tag3 in trigram_count.keys():
        trigram_val = trigram_count[(tag1,tag2,tag3)]
        if trigram_val > 0:
            trigram_p = (trigram_val)/(bigram_count[(tag2,tag3)])
            bigram_p = (bigram_count[(tag2,tag3)])/(unigram_count[(tag3,)])
            unigram_p = (unigram_count[(tag3,)])/total_sum
            ngram_list = [trigram_p,bigram_p,unigram_p]
            m = np.argmax(ngram_list)
            if m == 0:
                lambda3 += trigram_val
            if m == 1:
                lambda2 += trigram_val
            if m == 2:
                lambda1 += trigram_val
    
    weights = [lambda1,lambda2,lambda3]
    weight_sum = sum(weights)
    weights = [a / weight_sum for a in weights]
    
    return weights

    

In [14]:
weights = deleted_interpolation()
print(weights)
for tag1,tag2,tag3 in trigram_prob.keys():
    prob1 = trigram_prob[(tag1,tag2,tag3)]
    prob2 = bigram_prob[(tag2,tag3)]
    prob3 = unigram_prob[(tag3,)]
    prob1 = weights[0] * prob1
    prob2 = weights[1] * prob2
    prob3 = weights[2] * prob3
    total_prob = prob1 + prob2 + prob3
    trigram_prob[(tag1,tag2,tag3)] = total_prob
trigram_prob

[0.13629608860227113, 0.44765000700967333, 0.4160539043880555]


{('NOUN', '.', 'ADV'): -6.403414937119794,
 ('PRON', 'ADP', 'NUM'): -8.397903988784648,
 ('NOUN', 'PRON', '.'): -6.436907860126697,
 ('PRT', 'ADP', 'ADV'): -7.85884532743591,
 ('ADV', 'DET', 'PRT'): -9.890292640011186,
 ('NOUN', 'VERB', 'DET'): -4.881396329253624,
 ('DET', 'PRT', '.'): -7.3869728690970256,
 ('CONJ', 'NUM', 'VERB'): -7.990463835633557,
 ('ADP', 'ADP', 'DET'): -4.634081948441421,
 ('.', 'PRT', 'PRON'): -9.204591132065918,
 ('NUM', '.', 'DET'): -6.118730789542744,
 ('PRT', 'DET', '.'): -7.693882394377093,
 ('DET', 'CONJ', 'PRON'): -8.233629493198055,
 ('*', 'ADP', 'NUM'): -8.15060232132332,
 ('ADJ', 'ADP', 'PRON'): -6.334375486408418,
 ('VERB', 'PRT', 'PRT'): -9.003455204535959,
 ('DET', '.', 'X'): -12.316242624807817,
 ('*', '.', 'ADJ'): -7.164087879519445,
 ('*', 'CONJ', 'VERB'): -6.193783237334019,
 ('ADJ', 'ADP', 'NUM'): -8.121950470051077,
 ('DET', 'DET', 'VERB'): -6.335221609152237,
 ('ADV', 'NUM', 'ADJ'): -8.44845771629081,
 ('ADJ', 'NOUN', '.'): -3.98910163914897,

#### words which have count less than 5 are known words whereas those words with less than count of 5 are converted to __RARE__

In [15]:
known_words = set()
word_count = dict()
count1 = 0
for sentence in brown_words:
    for word in sentence:
        if word not in word_count.keys():
            word_count[word] = 0
        word_count[word] += 1
        count1 += 1

for word in word_count:
    if word_count[word] > 5:
        known_words.add(word)  


In [16]:
count = 0
brown_words_rare = brown_words
for i, sent_words in enumerate(brown_words):
    for j, word in enumerate(sent_words):
        if word not in known_words:
            brown_words_rare[i][j] = '__RARE__'
            count += 1

In [17]:
count1 = 0
count2 = 0
for i, sent_words in enumerate(brown_words_rare):
    for j, word in enumerate(sent_words):
        if brown_words_rare[i][j] == '__RARE__':
            count2 += 1
        count1 += 1
print(count1,count2)

625622 41260


In [18]:
word_given_tag_count = dict()
tag_count = dict()
pos_tag = zip(brown_words_rare,brown_tags)
count = 0

for sent_word,sent_tag in pos_tag:
#     print(sent_word,sent_tag)
    pos_tag1 = zip(sent_word,sent_tag)
    for word,tag in pos_tag1:
        if (word,tag) not in word_given_tag_count.keys():
            count += 1
            word_given_tag_count[(word,tag)] = 0
        word_given_tag_count[(word,tag)] += 1
        
        if tag not in tag_count.keys():
            tag_count[tag] = 0
        tag_count[tag] += 1

tag_class = set(tag_count)
word_given_tag_count

{('dig', 'VERB'): 8,
 ('room', 'NOUN'): 266,
 ('dull', 'ADJ'): 13,
 ('characteristics', 'NOUN'): 30,
 ('temperature', 'NOUN'): 106,
 ('force', 'NOUN'): 87,
 ('finish', 'VERB'): 15,
 ('deep', 'ADJ'): 38,
 ('talking', 'VERB'): 53,
 ('edythe', 'NOUN'): 14,
 ('jubal', 'NOUN'): 9,
 ('calendar', 'NOUN'): 21,
 ('wanting', 'VERB'): 10,
 ('tires', 'NOUN'): 6,
 ('concerns', 'NOUN'): 5,
 ('herd', 'NOUN'): 7,
 ('furnished', 'VERB'): 15,
 ('increase', 'NOUN'): 76,
 ('billy', 'NOUN'): 18,
 ('abrupt', 'ADJ'): 8,
 ('cliff', 'NOUN'): 6,
 ('parallel', 'NOUN'): 3,
 ('projects', 'NOUN'): 39,
 ("'", '.'): 108,
 ('stimulation', 'NOUN'): 8,
 ('total', 'VERB'): 2,
 ('staring', 'VERB'): 20,
 ('sing', 'VERB'): 12,
 ('escape', 'NOUN'): 10,
 ('underground', 'ADJ'): 8,
 ('methods', 'NOUN'): 81,
 ('grin', 'NOUN'): 7,
 ('sunrise', 'NOUN'): 7,
 ('advantages', 'NOUN'): 19,
 ('veteran', 'ADJ'): 1,
 ('noble', 'ADJ'): 9,
 ('concrete', 'NOUN'): 24,
 ('plants', 'NOUN'): 28,
 ('beef', 'NOUN'): 6,
 ('mud', 'NOUN'): 21,
 ('fo

### Emission Probabilities

In [19]:
prob_word_given_tag = dict()

for word,tag in word_given_tag_count:
    word_tag_count = word_given_tag_count[(word,tag)]
    count_tag = tag_count[tag]
    log_word_tag_count = math.log(word_tag_count,2)
    log_count_tag = math.log(count_tag,2)
    if (word,tag) not in prob_word_given_tag:
        prob_word_given_tag[(word,tag)] = 0
    prob_word_given_tag[(word,tag)] = log_word_tag_count - log_count_tag

prob_word_given_tag

{('dig', 'VERB'): -13.453881489107427,
 ('discovered', 'VERB'): -11.32459847216246,
 ('dull', 'ADJ'): -11.453824618783125,
 ('characteristics', 'NOUN'): -11.975012003802586,
 ('temperature', 'NOUN'): -10.153982144847907,
 ('force', 'NOUN'): -10.438959103562377,
 ('finish', 'VERB'): -12.546990893498908,
 ('deep', 'ADJ'): -9.906336823480633,
 ('talking', 'VERB'): -10.725961034544227,
 ('procurement', 'NOUN'): -12.63397508596752,
 ('calendar', 'NOUN'): -12.489585176632346,
 ('roman', 'ADJ'): -11.832336242036856,
 ('tires', 'NOUN'): -14.29694009868995,
 ('rolls', 'VERB'): -16.453881489107427,
 ('herd', 'NOUN'): -14.074547677353502,
 ('furnished', 'VERB'): -12.546990893498908,
 ('increase', 'NOUN'): -10.63397508596752,
 ('billy', 'NOUN'): -12.711977597968794,
 ('abrupt', 'ADJ'): -12.154264336924218,
 ('projects', 'NOUN'): -11.596500380548857,
 ('faced', 'VERB'): -12.061564066328668,
 ("'", '.'): -9.371877458078984,
 ('stimulation', 'NOUN'): -13.881902599411106,
 ('total', 'VERB'): -15.45388

In [20]:
pos_tag = zip(brown_words_rare,brown_tags)
tag_dict = dict()
for sent_word,sent_tag in pos_tag:
#     print(sent_word,sent_tag)
    pos_tag1 = zip(sent_word,sent_tag)
#     print(pos_tag1)
    for word,tag in pos_tag1:
#         print(word,tag)
        if tag not in tag_dict.keys():
            tag_dict[tag] = []
        if word not in tag_dict[tag]:
            tag_dict[tag].append(word)
tag_dict

{'*': ['*'],
 '.': ['.',
  ',',
  '(',
  ')',
  ':',
  '``',
  ';',
  "''",
  '--',
  "'",
  '?',
  '__RARE__',
  '!'],
 'ADJ': ['rough',
  'dirty',
  '__RARE__',
  'personal',
  'such',
  'state-owned',
  'other',
  'loose',
  'poor',
  'same',
  'past',
  'few',
  'many',
  'subsequent',
  'individual',
  'similar',
  'public',
  'rotary',
  'central',
  'recent',
  'new',
  'considerable',
  'actual',
  'prior',
  'uniform',
  'proper',
  'total',
  'annual',
  'executive',
  'current',
  'excessive',
  'present',
  'higher',
  'extensive',
  'outside',
  'much',
  'latter',
  'sound',
  'meaningful',
  'economical',
  'sufficient',
  'complete',
  'adequate',
  'general',
  'operational',
  'good',
  'necessary',
  'lowest',
  'responsible',
  'lower',
  'obvious',
  'federal',
  'special',
  'normal',
  'governmental',
  'important',
  'broad',
  'administrative',
  'chief',
  'local',
  'own',
  'several',
  'financial',
  'earlier',
  'municipal',
  'least',
  'additional',
  'r

## Reading test file

In [21]:
infile = open(cwd + '/Brown_dev.txt', 'r')
brown_dev = infile.readlines()
infile.close()

In [22]:
brown_dev_word = []
for sentence in brown_dev:
    brown_dev_word.append(sentence.split(' ')[:-1])
brown_dev_words = []
for sentence in brown_dev_word:
    sentence = [item.lower() for item in sentence]
    brown_dev_words.append(sentence)
brown_dev_words

[['he', 'went', 'to', 'the', 'market', 'to', 'buy', 'food']]

## Applying Viterbi Algo On The Test Data To get Pos Tagging

In [23]:
pi = dict()
back_pointer = {}
pi[(0,'*','*')] = 0.0
LOG_PROB_OF_ZERO = -1000
tagged = []
def S(k):
    if k in (-1,0):
        return {'*'}
    else:
        return tag_class

for given_words in brown_dev_words:
    sent_words = [word if word in known_words else '__RARE__' for word in given_words]
    n = len(sent_words)
    for k in range(1,n+1):
        for k in range(1, n+1):
            for u in S(k-1):
                for v in S(k):
                    max_score = float('-Inf')
                    max_tag = None
                    for w in S(k - 2):
                        if prob_word_given_tag.get((sent_words[k-1], v), 0) != 0:
                            score = pi.get((k-1, w, u), LOG_PROB_OF_ZERO) + trigram_prob.get((w, u, v), LOG_PROB_OF_ZERO) + \
                                    prob_word_given_tag.get((sent_words[k-1], v))
                            if score > max_score:
                                max_score = score
                                max_tag = w
                    pi[(k, u, v)] = max_score
                    back_pointer[(k, u, v)] = max_tag

        max_score = float('-Inf')
        u_max, v_max = None, None
        for u in S(n-1):
            for v in S(n):
                score = pi.get((n, u, v), LOG_PROB_OF_ZERO) + trigram_prob.get((u, v, 'STOP'), LOG_PROB_OF_ZERO)
                if score > max_score:
                    max_score = score
                    u_max = u
                    v_max = v

        tags = []
        tags.append(v_max)
        tags.append(u_max)

        for i, k in enumerate(range(n-2, 0, -1)):
            tags.append(back_pointer[(k+2, tags[i+1], tags[i])])
        tags.reverse()

        tagged_sentence = []
    for j in range(0, n):
        tagged_sentence.append(given_words[j] + '/' + tags[j])
    tagged_sentence.append('\n')
    tagged.append(' '.join(tagged_sentence))


In [24]:
tagged

['he/PRON went/VERB to/ADP the/DET market/NOUN to/PRT buy/VERB food/NOUN \n']

In [25]:
for i in tagged:
    i = i.split()
    for j in i:
        
        print(j)# for a in prob_word_given_tag.key

he/PRON
went/VERB
to/ADP
the/DET
market/NOUN
to/PRT
buy/VERB
food/NOUN


In [26]:
input_arr = []
for tag in tagged:
#     print(tag)
    t = tag.split()
    for word in t:
        word = word.split('/')
        input_arr.append(word[1])
input_arr

['PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'PRT', 'VERB', 'NOUN']

### print out the top 50 words (most probable) emitted by it. 



In [27]:
top50 = dict()
for tag in input_arr:
    words_list = tag_dict[tag]
    ans = []
    prob_list = []
    for word in words_list:
        current_prob = prob_word_given_tag[(word,tag)]
        prob_list.append((word,current_prob))
    sorted_list = sorted(prob_list,key = itemgetter(1),reverse=True)
#     print(sorted_list)
    ans = sorted_list[:50]
    final_ans = []
    for word in ans:
        word = word[0]
        final_ans.append(word)
#     print(final_ans)
    if tag not in top50.keys():
        top50[tag] = []
    top50[tag] = final_ans
    
top50

{'ADP': ['of',
  'in',
  'to',
  'for',
  'with',
  'on',
  'that',
  'as',
  'at',
  'by',
  'from',
  'if',
  'into',
  'than',
  'like',
  'about',
  'through',
  'after',
  'over',
  'before',
  'between',
  'under',
  'because',
  'while',
  'against',
  'so',
  'since',
  'during',
  'without',
  'until',
  'out',
  'upon',
  'toward',
  'though',
  'around',
  'within',
  'across',
  'behind',
  'although',
  'down',
  'among',
  'per',
  'whether',
  'up',
  'above',
  'except',
  'along',
  'near',
  'off',
  '-'],
 'DET': ['the',
  'a',
  'his',
  'this',
  'an',
  'which',
  'her',
  'that',
  'their',
  'what',
  'no',
  'my',
  'these',
  'some',
  'its',
  'any',
  'each',
  'our',
  'your',
  'those',
  'both',
  'another',
  'every',
  'whose',
  'whatever',
  'either',
  'neither',
  '__RARE__',
  'them',
  'one'],
 'NOUN': ['__RARE__',
  'af',
  'time',
  'man',
  'way',
  'state',
  'years',
  'men',
  'day',
  'eyes',
  'people',
  'one',
  'something',
  'hand',
  

## Implementation of Forward Backward Procedure to  find the probability of the observation sequence

In [28]:
    seq = 'One man remarked that if he had a hundred pounds , he would give ninety of them to be back in England .'
### Forwarding 

    seq_words = seq.split(' ')
    seq_len = len(seq_words)
#     states = unigram_count.keys()
    alpha = [{}]
    seq = seq_words
    for state in tag_class:
        state_prob = 1
        emit_prob = 1
        if (state,) in unigram_count:
            state_prob = math.log(unigram_count[(state,)]/float(sum(unigram_count.values())))
#             print(state_prob)
        if (seq[0],state) not in prob_word_given_tag:
            emit_prob = 1
        else:
            emit_prob = prob_word_given_tag[(seq[0],state)]
        alpha[0][state] = state_prob + emit_prob
#         print(alpha[0][state])

    for i in range(1,seq_len):
        alpha.append({})
        for state_to in tag_class:
            prob = 1
            emit_prob = 1
            for state_from in tag_class:
                trans_prob = 1
#                 print((state_from,state_to))
                if (state_from,state_to) in bigram_prob:
                    trans_prob = bigram_prob[(state_from,state_to)]
                else:
                    trans_prob = 1
                
                prob = alpha[i-1][state_from] + trans_prob
#                 print(prob)
            if (state_to,seq[i]) in prob_word_given_tag:
                emit_prob = prob_word_given_tag[(state_to,seq[i])]
            else:
                emit_prob = 1
#             print(emit_prob)
            alpha[i][state_to] = prob + emit_prob
    alpha



[{'*': -1.431740495913401,
  '.': -1.168279970090354,
  'ADJ': -1.84236603523618,
  'ADP': -1.2511998942971974,
  'ADV': -2.1179278484736845,
  'CONJ': -2.5607283693187584,
  'DET': -1.293011836248796,
  'NOUN': -0.6448584445659089,
  'NUM': -3.5420264499886516,
  'PRON': -2.134463773043123,
  'PRT': -2.6900661776650874,
  'STOP': -2.124887676473346,
  'VERB': -0.9415400703930425,
  'X': -5.966379096935564},
 {'*': -0.13446377304312307,
  '.': -8.686582435404118,
  'ADJ': -12.376198507597293,
  'ADP': -9.81380157126408,
  'ADV': -9.814790734855032,
  'CONJ': -11.989175384488046,
  'DET': -11.376198507597293,
  'NOUN': -12.330212302323112,
  'NUM': -15.680709549407247,
  'PRON': -12.848832308215574,
  'PRT': -10.904605561334082,
  'STOP': -0.13446377304312307,
  'VERB': -6.1171700462350564,
  'X': -20.32456573918197},
 {'*': -10.848832308215574,
  '.': -19.400950970576567,
  'ADJ': -23.09056704276974,
  'ADP': -20.52817010643653,
  'ADV': -20.529159270027485,
  'CONJ': -22.7035439196604

In [29]:
    ### backwarding
    seq = 'One man remarked that if he had a hundred pounds , he would give ninety of them to be back in England .'
    seq_words = seq.split(' ')
    seq_len = len(seq_words)
    beta = [{}]
    seq = seq_words
    
    for state in tag_class:
        beta[0][state] = 1
    for i in range(seq_len - 1, 0, -1):
        beta.insert(0,{})
        for state_from in tag_class:
            prob = 0
            for state_to in tag_class:
                trans_prob = 0
                if (state_from,state_to) in bigram_prob:
                        trans_prob = bigram_prob[(state_from,state_to)]
                emit_prob = 0
                if (state_to,seq[i]) in prob_word_given_tag:
                    emit_prob = prob_word_given_tag[(state_to,seq[i])]
                prob = beta[1][state_to]+ trans_prob + emit_prob
            beta[0][state_from] = prob
    beta




[{'*': -251.81029825357564,
  '.': -252.25593952249264,
  'ADJ': -256.69998810843055,
  'ADP': -251.75877743963815,
  'ADV': -253.62578716658913,
  'CONJ': -253.67417181662643,
  'DET': -254.92505466406533,
  'NOUN': -253.01379887589542,
  'NUM': -258.5194158627887,
  'PRON': -256.71610777379385,
  'PRT': -257.13655876925907,
  'STOP': -245.00173923862138,
  'VERB': -251.64446464126212,
  'X': -263.19184120476024},
 {'*': -240.0959297184032,
  '.': -240.5415709873202,
  'ADJ': -244.9856195732581,
  'ADP': -240.0444089044657,
  'ADV': -241.9114186314167,
  'CONJ': -241.959803281454,
  'DET': -243.2106861288929,
  'NOUN': -241.29943034072298,
  'NUM': -246.8050473276163,
  'PRON': -245.00173923862138,
  'PRT': -245.4221902340866,
  'STOP': -233.28737070344894,
  'VERB': -239.93009610608968,
  'X': -251.4774726695878},
 {'*': -228.38156118323076,
  '.': -228.82720245214776,
  'ADJ': -233.27125103808567,
  'ADP': -228.33004036929327,
  'ADV': -230.19705009624425,
  'CONJ': -230.24543474628

In [30]:
seq = 'The anchor cable would have been lost overboard'
seq_words = seq.split(' ')
seq_len = len(seq_words)
# print (seq_len)
prob = 0
for state in alpha[seq_len - 1]:
    prob += alpha[seq_len - 1][state]
print (prob)


-1042.5387271484751
